In [1]:
import pickle
import pandas as pd
import numpy as np

In [3]:
# Load saved models

# Load PD model
pd_ensemble = pickle.load(open('pd_ensemble_model.sav', 'rb'))

# Load LGD stage 1
rf_classifier = pickle.load(open('rf_classifier_model_lgd_stage_1.sav', 'rb'))

# Load LGD stage 2

reg_lgd_st_2 = pickle.load(open('lgd_model_stage_2.sav', 'rb'))

# Load EAD model

reg_ead = pickle.load(open('ead_model_.sav', 'rb')) 

In [4]:
# Load data to use with the PD model. Loading both train and test sets.

loan_data_inputs_train = pd.read_pickle('loan_data_inputs_train.pkl')

loan_data_inputs_test = pd.read_pickle('loan_data_inputs_test.pkl')

In [5]:
# Concatenate both dataframes into one dataframe. 
# Use axis = 0 to join by rows

loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)



In [6]:
# check
loan_data_inputs_pd.shape

(466285, 322)

In [7]:
loan_data_inputs_pd.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,1,0,0,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,0,0,0,1,0,0,0,0,0,0
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,0,0,1,0,0,0,0,0,0
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,0,0,0,1,0,0,0,0,0,0
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,0,1,0,1,0,0,0,0,0,0


In [12]:
# All features used to train the PD model
features_all_pd = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86']

In [13]:
# reference categories with the PD model
ref_categories_pd = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [14]:
# Keep only the variables used to train PD model
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]


In [15]:
# Remove the reference categories

loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd, axis = 1)


In [16]:
# check. Matches with the shape of dataframe used to train the PD model
loan_data_inputs_pd_temp.shape

(466285, 84)

In [18]:
# Use PD model to predict probability of default

loan_data_inputs_pd['PD'] = pd_ensemble.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]


In [19]:
#check
loan_data_inputs_pd['PD'].head()

427211    0.029216
206088    0.113854
136020    0.035900
412305    0.197927
36159     0.202047
Name: PD, dtype: float64

In [20]:
# check descriptive statistics
loan_data_inputs_pd['PD'].describe()

count    466285.000000
mean          0.108792
std           0.070185
min           0.007477
25%           0.056088
50%           0.093216
75%           0.145914
max           0.577053
Name: PD, dtype: float64

In [21]:
# Load pre-processed dataset for using with LGD and EAD models



loan_data_preprocessed = pd.read_pickle('loan_data_2007_2014_preprocessed.pkl')

In [22]:
# check

loan_data_preprocessed.head()



,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,0,0,0,0,0,0,0,0,1,0
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,0,0,0,0,0,0,0,1,0
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,0,0,0,0,0,0,0,1,0
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,0,0,0,0,0,0,0,1,0
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,0,0,0,0,0,0,0,1,0


In [23]:
#check 
loan_data_preprocessed.shape

(466285, 207)

In [24]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Impute missing values using IterativeImputer (MICE)
imputer = IterativeImputer(random_state=42)
loan_data_preprocessed[['mths_since_last_delinq', 'mths_since_last_record']] = imputer.fit_transform(loan_data_preprocessed[['mths_since_last_delinq', 'mths_since_last_record']])

In [25]:
# Check if missing values are filled
print(loan_data_preprocessed[['mths_since_last_delinq', 'mths_since_last_record']].isnull().sum())

mths_since_last_delinq    0
mths_since_last_record    0
dtype: int64


In [26]:
#features used for LGD and EAD models

features_all = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']

In [27]:
# List contianing reference variables used with LGD and EAD models
features_reference_cat = ['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

In [28]:
# keep only variables used by EAD and LGD models

loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]


In [29]:

loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]# Remove reference variables

loan_data_preprocessed_lgd_ead = loan_data_preprocessed_lgd_ead.drop(features_reference_cat, axis = 1)


In [30]:
# Apply LGD stage 1 model

loan_data_preprocessed['recovery_rate_st_1'] = rf_classifier.predict(loan_data_preprocessed_lgd_ead)


In [31]:
# Applt LGD stage 2 model

loan_data_preprocessed['recovery_rate_st_2'] = reg_lgd_st_2.predict(loan_data_preprocessed_lgd_ead)


In [32]:
# Combine the predicted values from the stage 1 predicted model and the stage 2 predicted model
# to calculate the final estimated recovery rate.
# Same as done before: recovery_rate = recovery_rate_st_1 * recovery_rate_st_2

loan_data_preprocessed['recovery_rate'] = loan_data_preprocessed['recovery_rate_st_1'] * loan_data_preprocessed['recovery_rate_st_2']


In [33]:
loan_data_preprocessed['recovery_rate'].describe()

count    466285.000000
mean          0.046253
std           0.055339
min          -0.014573
25%           0.000000
50%           0.000000
75%           0.101968
max           2.316704
Name: recovery_rate, dtype: float64

In [34]:
# Set estimated recovery rates that are greater than 1 to 1 and  estimated recovery rates that are less than 0 to 0

loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] < 0, 0, loan_data_preprocessed['recovery_rate'])
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] > 1, 1, loan_data_preprocessed['recovery_rate'])


In [35]:
# Compute LGD 
# LGD = 1 - recovery_rate

loan_data_preprocessed['LGD'] = 1 - loan_data_preprocessed['recovery_rate']


In [36]:
# check
loan_data_preprocessed['LGD'].describe()


count    466285.000000
mean          0.953749
std           0.055257
min           0.000000
25%           0.898032
50%           1.000000
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

In [37]:
# Use EAD model to predict credit conversion factor

loan_data_preprocessed['CCF'] = reg_ead.predict(loan_data_preprocessed_lgd_ead)


In [38]:
# check
loan_data_preprocessed['CCF'].describe()

count    466285.000000
mean          0.746351
std           0.079318
min           0.437529
25%           0.702739
50%           0.739980
75%           0.777174
max           1.114038
Name: CCF, dtype: float64

In [39]:
# Compute EAD using CCF
# EAD = CCF * funded_amt

loan_data_preprocessed['EAD'] = loan_data_preprocessed['CCF'] * loan_data_preprocessed_lgd_ead['funded_amnt']


In [40]:
# check
loan_data_preprocessed['EAD'].describe()

count    466285.000000
mean      10943.520004
std        6894.175605
min         228.698297
25%        5632.390900
50%        9310.382776
75%       14952.275470
max       34108.113322
Name: EAD, dtype: float64

In [41]:
# LGD and EAD is in this dataframe
loan_data_preprocessed.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,0,0,1,0,1,0.086851,0.086851,0.913149,0.624418,3122.091474
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,0,1,0,1,0.086416,0.086416,0.913584,0.753184,1882.960861
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,0,1,0,1,0.080708,0.080708,0.919292,0.636223,1526.935546
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,0,1,0,1,0.094859,0.094859,0.905141,0.636153,6361.529597
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,0,1,0,1,0.091036,0.091036,0.908964,0.751931,2255.791594


In [42]:
# This data frame has the PD.
loan_data_inputs_pd.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,0,0,1,0,0,0,0,0,0,0.029216
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,0,0,1,0,0,0,0,0,0,0.113854
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,0,1,0,0,0,0,0,0,0.035900
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,0,0,1,0,0,0,0,0,0,0.197927
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,1,0,1,0,0,0,0,0,0,0.202047


In [43]:
# check shape of both dataframes

print (loan_data_preprocessed.shape)
print (loan_data_inputs_pd.shape)

(466285, 213)
(466285, 323)


In [44]:
# Concatenate the dataframes where we calculated LGD and EAD and the dataframe where we calculated PD along the columns.
loan_data_preprocessed_new = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)

In [45]:
# check
loan_data_preprocessed_new.shape

(466285, 536)

In [46]:
loan_data_preprocessed_new.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,1,0,1,0,0,0,0,0,0,0.164521
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,0,1,0,0,0,0,0,0,0.281159
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,0,1,0,0,0,0,0,0,0.221345
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,0,1,0,0,0,0,0,0,0.208483
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,0,1,0,0,0,0,0,0,0.130858


### compute Expected loss

### Expected Loss. EL = PD * LGD * EAD.

In [47]:
loan_data_preprocessed_new['EL'] = loan_data_preprocessed_new['PD'] * loan_data_preprocessed_new['LGD'] * loan_data_preprocessed_new['EAD']


In [48]:
#check 
loan_data_preprocessed_new['EL'].describe()

count    466285.000000
mean       1106.927448
std        1077.120259
min           0.000000
25%         378.770943
50%         743.619427
75%        1454.147209
max       12320.544027
Name: EL, dtype: float64

In [49]:
# check the computed columns
loan_data_preprocessed_new[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head()

,funded_amnt,funded_amnt,PD,LGD,EAD,EL
0,5000,5000,0.164521,0.913149,3122.091474,469.039688
1,2500,2500,0.281159,0.913584,1882.960861,483.661654
2,2400,2400,0.221345,0.919292,1526.935546,310.701632
3,10000,10000,0.208483,0.905141,6361.529597,1200.462358
4,3000,3000,0.130858,0.908964,2255.791594,268.316223


Bank these days have to report their EL for all their loans 

They are also required by regulators to show that they have enough cash to cover their EL.

Therefore, we compute the agrregated EL for all loans


In [50]:
# Total expected loss (EL). In this case its around 501 million.

loan_data_preprocessed_new['EL'].sum()

516143665.14323455

In [51]:
# Total loan given by bank. In this case its around 6.6 billon'

loan_data_preprocessed_new['funded_amnt'].sum()

funded_amnt    6664052450
funded_amnt    6664052450
dtype: int64

### Banks are generally required to have EL to funded_amt ratio to be between 2% and 10 %.

In [52]:
el_to_amt_funded = loan_data_preprocessed_new['EL'].sum() / loan_data_preprocessed_new['funded_amnt'].sum()
print ('EL/Total funded amount', el_to_amt_funded)

EL/Total funded amount funded_amnt    0.077452
funded_amnt    0.077452
dtype: float64


### In this case the bank's EL is around 7.7% of the total loans on its book.

### Is the bank taking too much risk?

Just Right.